# BaseLines:User-based CF& Time-Decay CF

In [5]:
#-*- coding:UTF-8 -*-
import numpy as np
import pandas as pd
import operator
from operator import itemgetter

import prettyplotlib as ppl




import matplotlib.pyplot as plt
import matplotlib as mpl
from prettyplotlib import brewer2mpl

"""TARP推荐最新版本 """
def userBasedRecommend(neighbors,userId,users,K,TopN,ShowWatched):
    movieRatings={}
    DIVmovieRatings={}
    #print(neighbors)
    for n in neighbors:#(0.7, '小明')
        sim=n[0]#0.7
        movieDic=users[n[1]]#每个用户的全部评价字典{'movie':1,'movie2':2...}
        for m in movieDic:#物品ID
            movieDic[m]
            if m in movieRatings: #如果已经在预备推荐的列表中了
                movieRatings[m]=movieDic[m]*sim+movieRatings[m]#加上之前的值
                DIVmovieRatings[m]=DIVmovieRatings[m]+1
            else:#如果预备推荐列表中没有
                movieRatings[m]=movieDic[m]*sim
                #DIVmovieRatings[m]=sim
                DIVmovieRatings[m]=1
                

    """如果每个用户对物品评分都是一样的话movieRatings[m]/DIVmovieRatings[m]都是1了  """
    for m in movieRatings:
        movieRatings[m]=movieRatings[m]/DIVmovieRatings[m]
  

    mylist=sorted(movieRatings.items(),key=lambda x:x[1],reverse=True)
    #print("mylist",mylist)
    if ShowWatched==False:# 不推荐已经看过的
        for m in mylist:
            if m[0] in users[userId].keys():
                mylist.remove(m)
    
    return mylist[:TopN]

"""给一个user生成推荐RPTARS """                    
def userBasedRecommendTA(alfa,beta,neighbors,userId,users,usersTA,K,TopN,ShowWatched,testAUserItems,typeName):
    movieRatings={}
    DIVmovieRatings={}
    #每个邻居
    for n in neighbors:#(0.7, '小明')
        sim=n[0]#0.7
        movieDic=users[n[1]]#每个用户的全部评价字典{'movie':1,'movie2':2...}
        #每个邻居的每个item
        for m in movieDic:#  item:1 每个邻居的每个item. m is key
            movieDic[m]#分数   1
            if m in movieRatings:#not first item, normalization
                movieRatings[m]=movieDic[m]*sim+movieRatings[m]
                #DIVmovieRatings[m]=sim+DIVmovieRatings[m]
                DIVmovieRatings[m]=DIVmovieRatings[m]+1
            else:  # if first item
                movieRatings[m]=movieDic[m]*sim   #预测分数
                #DIVmovieRatings[m]=sim             #相似度
                DIVmovieRatings[m]=1


    for m in movieRatings: #每个生成的预测分数
        movieRatings[m]=movieRatings[m]/DIVmovieRatings[m]
        dictvalues=list(testAUserItems.values())#[[16604.67078275463]]#time now
        #print("testAUserItems",)        
        #乘以个TimeFunction
        if(typeName=='TARP'):
            timeFunction=RPTimeFuction(alfa,beta,usersTA[userId],m,dictvalues[0][0])
            
          
        elif(typeName=='TDF'):
            #timeFunction=TDFTimeFuction(usersTA[userId],m,dictvalues[0][0])
            
            timeFunction=TDFTimeFunctionTImeDecay(usersTA[userId],m,dictvalues[0][0])
        movieRatings[m]=movieRatings[m]*(1+timeFunction)
    mylist=sorted(movieRatings.items(),key=lambda x:x[1],reverse=True)
    if ShowWatched==False:# 不推荐已经看过的
        for m in mylist:
            if m[0] in users[userId].keys():
                mylist.remove(m)
    return mylist[:TopN]


reparchaseCycleList=[]
timeSetList=[]
rPCList=[]
#时间函数
def RPTimeFuction(alfa,beta,aUsersItemTimes,itemKey,NowTimestampTestSet):
   
    daltaTsum=0
    preTime=-1
    BaseP=beta
    #NowTimestampTestSet=15316
    #NowTimestampTestSet=15302
    #NowTimestampTestSet=15272
    #NowTimestampTestSet=15202
    if itemKey not in aUsersItemTimes.keys():#判断是否买过
        return BaseP   
    aItemTimes=aUsersItemTimes[itemKey]#aItemTimes:[100,200,300]
    prechaseTimes=len(aItemTimes)
    for time in aItemTimes:
        if preTime==-1:#第一次循环的路径
            preTime=time
        else:#之后循环操作
            daltaTsum=daltaTsum+abs(preTime-time)
            preTime=time
    if(prechaseTimes<2):
        return BaseP
    else:
        reparchaseCycle=(daltaTsum/(prechaseTimes-1))
    
    if(reparchaseCycle!=0):
        reparchaseCycleList.append(NowTimestampTestSet-aItemTimes[-1]-reparchaseCycle)
        timeSetList.append(NowTimestampTestSet-aItemTimes[-1])
        rPCList.append(reparchaseCycle)
    if reparchaseCycle==0:
        return BaseP
    result=(1-BaseP)*np.exp(-(np.square(NowTimestampTestSet+0*reparchaseCycle+TimeDelay-aItemTimes[-1]-reparchaseCycle))/(alfa*alfa*reparchaseCycle*reparchaseCycle))+BaseP
    return  result
def RPTimeFuctionNN(alfa,beta,aUsersItemTimes,itemKey,NowTimestampTestSet):
    daltaTsum=0
    preTime=-1
    BaseP=beta
   
    if itemKey not in aUsersItemTimes.keys():#判断是否买过
        return BaseP   
    aItemTimes=aUsersItemTimes[itemKey]#aItemTimes:[100,200,300]
    aItemQuantity=aUsersItemQuantity[itemKey]
    prechaseTimes=len(aItemTimes)
    if(prechaseTimes<2):
        return BaseP
    else:
        reparchaseCycle=predictSingle(userId,itemKey,aItemTimes[-1],aItemQuantity[-1])  
    if(reparchaseCycle!=0):
        reparchaseCycleList.append(NowTimestampTestSet-aItemTimes[-1]-reparchaseCycle)
        timeSetList.append(NowTimestampTestSet-aItemTimes[-1])
        rPCList.append(reparchaseCycle)    
    
   
   
    
    result=(1-BaseP)*np.exp(-(np.square(NowTimestampTestSet+0*reparchaseCycle+TimeDelay-aItemTimes[-1]-reparchaseCycle))/(alfa*alfa*reparchaseCycle*reparchaseCycle))+BaseP
    
    return result        
#TDF时间函数
def TDFTimeFuction(aUsersItemTimes,itemKey,NowTimestampTestSet):
    BaseP=1
    daltaTsum=0
    preTime=-1   
    print(BaseP)
    if itemKey not in aUsersItemTimes.keys():#判断是否买过，没买过就返回1
        print(BaseP)
        return BaseP   
    aItemTimes=aUsersItemTimes[itemKey]#aItemTimes:[   100,200,300]
    prechaseTimes=len(aItemTimes)
    
   
    print("prechaseTimes",prechaseTimes)
    if(prechaseTimes<2):
        return BaseP
    else:
        reparchaseCycle=(daltaTsum/(prechaseTimes-1))    
    result=np.exp(-0.0001*(NowTimestampTestSet+0*reparchaseCycle-aItemTimes[-1]))
    print("result",result)     
    return result 
def TDFTimeFunctionTImeDecay(aUsersItemTimes,itemKey,NowTimestampTestSet):
    daltaTsum=0
    preTime=-1
    BaseP=1
    if itemKey not in aUsersItemTimes.keys():#判断是否买过
        return BaseP   
    aItemTimes=aUsersItemTimes[itemKey]#aItemTimes:[100,200,300]
    prechaseTimes=len(aItemTimes)
    for time in aItemTimes:
        if preTime==-1:#第一次循环的路径
            preTime=time
        else:#之后循环操作
            daltaTsum=daltaTsum+abs(preTime-time)
            preTime=time
    if(prechaseTimes<2):
        return BaseP
    else:
        reparchaseCycle=(daltaTsum/(prechaseTimes-1))
    result=np.exp(-0.005*(NowTimestampTestSet+0*reparchaseCycle+TimeDelay-aItemTimes[-1]))
    return result


def userBasedRecommendAll(AllNeibors,users,K,TopN,ShowWatched,testUseritems):
    rightNum=0
    hasBuyNum=0

   
    #return recall,precision    
    for userid in users.keys():
        
        topNRecommenderList=userBasedRecommend(AllNeibors[userid],userid,users,K,TopN,ShowWatched)
        for useritem in topNRecommenderList:
            itemid=useritem[0]
            try:#如果测试集有这个用户
                if itemid in testUseritems[userid].keys():
                    rightNum=rightNum+1
                    rightNumUser=rightNumUser+1
                #判断是否推荐的内容在用户买过的列表中
                if itemid in users[userid].keys():
                    hasBuyNum=hasBuyNum+1                
            except:
                #print("non")
                continue
    userNum=len(users)       
    precision=rightNum/(TopN*userNum)
    #recall=rightNum/len(testUseritems)
    recall=rightNum/(userNum*3)
    F1=2*precision*recall/(precision+recall)
    #print("userNum=",len(users))
    print("CF:")
    print ("precision=",precision,"rightNum",rightNum,"recommend",TopN*len(users))
    print ("recall=",recall,"testSetNum",len(testUseritems))
    print ("F1=",F1)
    Novelty=hasBuyNum/(TopN*userNum)
    print("Novelt=",Novelty)    
"""给每个user生成推荐并计算准确率 """       
def userBasedRecommendAllTARP(alfa,beta,AllNeibors,users,usersTA,K,TopN,ShowWatched,testUseritems,testUseritemsTA):
    rightNum=0
    hasBuyNum=0#推荐中已经买过的数量
    xunhuanNum=0
    userList= sorted(users.keys())
    
    for userid in userList:
        
        topNRecommenderList=userBasedRecommendTA(alfa,beta,AllNeibors[userid],userid,users,usersTA,K,TopN,ShowWatched,testUseritemsTA[userid],"TARP")
        #便利推荐的内容
        for useritem in topNRecommenderList:#same
            
            #print("useritem",useritem)
            itemid=useritem[0]    
            
            try:
                #print("testUseritems[userid]",testUseritems[userid])
                #print("userid",userid,"testUseritems[userid]",testUseritems[userid])
                #判断是否推荐的内容在测试集中
                if itemid in testUseritems[userid].keys():
                   # print("INuserId",userid,"itemid",itemid)
                    xunhuanNum=xunhuanNum+1
                    rightNum=rightNum+1
                #else:
                   # print("NotInuserId",userid,"itemid",itemid)
                #判断是否推荐的内容在用户买过的列表中
                if itemid in users[userid].keys():
                    hasBuyNum=hasBuyNum+1
                   
            except:
                print("non")
                #same 0
                continue
    print("rightNumxunhuanNum",xunhuanNum)
    userNum=len(users)       
    precision=rightNum/(TopN*userNum)
  
    recall=rightNum/(userNum*3)
   
    print("testUseritems",len(testUseritems),"user*3",userNum*3)
    F1=2*precision*recall/(precision+recall)
    print ("TARP:")
    print ("precision=",precision,"rightNum",rightNum,"recommend",TopN*len(users))    
    print ("recall=",recall)
    print ("F1=",F1)
    
    Novelty=hasBuyNum/(TopN*userNum)
    print("hasBuyNum:",hasBuyNum,"TopN:",TopN,"userNum:",userNum)
    print("Novelt=",Novelty)
    return 
    


"""时间下降曲线 """       
def userBasedRecommendAllTDF(AllNeibors,users,usersTA,K,TopN,ShowWatched,testUseritems,testUseritemsTA):
    rightNum=0
    hasBuyNum=0
    
    
    #遍历所有user 并生成推荐
    for userid in users.keys():
        
        topNRecommenderList=userBasedRecommendTA(0,0,AllNeibors[userid],userid,users,usersTA,K,TopN,ShowWatched,testUseritemsTA[userid],"TDF")
        
        #便利推荐的内容
        for useritem in topNRecommenderList:
            itemid=useritem[0]      
            try:
                if itemid in testUseritems[userid].keys():
                    rightNum=rightNum+1
               
                    
                if itemid in users[userid].keys():
                    hasBuyNum=hasBuyNum+1                   
            except:
                #print("non")
                continue
       
    userNum=len(users)         
    precision=rightNum/(TopN*userNum)
    recall=rightNum/(userNum*3)
    F1=2*precision*recall/(precision+recall)
    print ("TDF:")
    print ("precision=",precision,"rightNum",rightNum,"recommend",TopN*len(users))    
    print ("recall=",recall,"testsetNum",len(testUseritems))
    print ("F1=",F1)
    Novelty=hasBuyNum/(TopN*userNum)
    print("Novelt=",Novelty)    
    return 
    

def getNeighborsFormFile():
    FilePath="processeddata/AllNeigborsCos.csv"
    rating=pd.read_csv(FilePath)
    AllNeighbors={}#{"user1":[(1,老王),(0.74,老李)],"user2":}
    preUserId=0
    dictList=[] #[(1,老王),(0.74,老李)]
    for index,userId,similar,neighborId in rating.values:
        
        if userId==preUserId:# same user
            dictList.append((similar,neighborId))
        else: #new user
            if preUserId==0: #first user
                dictList.append((similar,neighborId))
            else: # new user
                AllNeighbors[preUserId]=dictList #add preverious user
                
                dictList=[]
                dictList.append((similar,neighborId))
    
        preUserId=userId
    AllNeighbors[preUserId]=dictList #last user 
    #print (AllNeighbors)
    return AllNeighbors


"""读取无时间的数据"""
def loadUserRatingData(rating):
    preUserId=-1
    userMovieDic={}
    ratingdict={}#每个用户各自的打分
    for index2,InvoiceNo,itemid,Description,Quantity,timestamp,userId in rating.values:
        
        #如果第一次循环
        if preUserId==-1:
            ratingdict[itemid]=1
            userMovieDic[userId]=ratingdict
        else:
            #如果接下来还是上一个用户的
            if(userId==preUserId):
                ratingdict[itemid]=1
            #如果接下来是新用户的
            if(userId!=preUserId):
                #把上个用户的信息存到大字典里
                userMovieDic[preUserId]=ratingdict
                #清空上个用户信息
                ratingdict={}#清空
                #保存这个用户信息
                ratingdict[itemid]=1
        #赋当前用户名值给旧用户名
        preUserId=userId
    #print(userMovieDic)

    return userMovieDic

def loadUserRatingDataTime(rating,dataset):
    preUserId=-1
    
    userMovieDic={}
    ratingdict={}#每个用户各自的打分
    timeList=[]#每个用户对应每个物品的打分
    alltimeListInDay=[]
    for  index,InvoiceNo,itemid,Description,Quantity,timestamp,userId in rating.values:
        #如果第一次循环
        #timeArray = time.strptime(timestamp, "%m/%d/%Y %H:%M")
        #转换为时间戳:
        #timestamp = int(time.mktime(timeArray))
        #userMovieDic={"user1":{"item1":[10d,100d,200d],"item2":[10d,20d]},[]}    
        timestamp=timestamp/3600/24
        alltimeListInDay.append(timestamp)
        #print("timeStamp",timestamp) 
        if preUserId==-1:
            
            timeList.append(timestamp)
            ratingdict[itemid]=timeList
            userMovieDic[userId]=ratingdict
        else:
            #如果接下来还是上一个用户的
            if userId==preUserId :
                if itemid in ratingdict.keys():#if user's item list already have this item
                    timeList=ratingdict[itemid]#append the time ti time list
                else: # user's item list do not have this item yet
                    timeList=[]#clear the time list
                timeList.append(timestamp)
                ratingdict[itemid]=timeList
                
            #如果接下来是新用户的
            if userId!=preUserId :
                
                
                #把上个用户的信息存到大字典里
                userMovieDic[preUserId]=ratingdict
                #清空上个用户信息
                ratingdict={}#清空
                timeList=[]
                #保存这个用户信息
                timeList.append(timestamp)
                ratingdict[itemid]=timeList                
                
        #赋当前用户名值给旧用户名
        preUserId=userId
        
    #print(userMovieDic)
    if dataset=='test':#测试集输出时间分布
        #ppl.boxplot(alltimeListInDay)
        
        #plt.show()        
        print("test time mean:",np.mean(alltimeListInDay))
    return userMovieDic

def loadUserRatingQuantity(rating):
    preUserId=-1
    
    userMovieDic={}
    ratingdict={}#每个用户各自的打分
    quantityList=[]#每个用户对应每个物品的打分
    allquantityListInDay=[]
    for  index,InvoiceNo,itemid,Description,Quantity,timestamp,userId in rating.values:
        allquantityListInDay.append(Quantity)
        if preUserId==-1:
            quantityList.append(Quantity)
            ratingdict[itemid]=quantityList
            userMovieDic[userId]=ratingdict
        else:
            #如果接下来还是上一个用户的
            if userId==preUserId :
                if itemid in ratingdict.keys():#if user's item list already have this item
                    quantityList=ratingdict[itemid]#append the time ti time list
                else: # user's item list do not have this item yet
                    quantityList=[]#clear the time list
                quantityList.append(Quantity)
                ratingdict[itemid]=quantityList
                
            #如果接下来是新用户的
            if userId!=preUserId :
                
                
                #把上个用户的信息存到大字典里
                userMovieDic[preUserId]=ratingdict
                #清空上个用户信息
                ratingdict={}#清空
                quantityList=[]
                #保存这个用户信息
                quantityList.append(Quantity)
                ratingdict[itemid]=quantityList                
                
        #赋当前用户名值给旧用户名
        preUserId=userId
        
    print(userMovieDic)
    
    return userMovieDic

"""读取无时间的数据"""
def loadUserRatingDataFromMovL(rating):
    preUserId=-1
    userMovieDic={}
    ratingdict={}#每个用户各自的打分
    for index2,userId,itemid,rating,timestamp in rating.values:
        #如果第一次循环
        if preUserId==-1:
            ratingdict[itemid]=rating
            userMovieDic[userId]=ratingdict
        else:
            #如果接下来还是上一个用户的
            if(userId==preUserId):
                ratingdict[itemid]=rating
            #如果接下来是新用户的
            if(userId!=preUserId):
                #把上个用户的信息存到大字典里
                userMovieDic[preUserId]=ratingdict
                #清空上个用户信息
                ratingdict={}#清空
                #保存这个用户信息
                ratingdict[itemid]=rating
        #赋当前用户名值给旧用户名
        preUserId=userId
    #print(userMovieDic)

    return userMovieDic

def loadUserRatingDataTimeFromMovL(rating):
    preUserId=-1
    
    userMovieDic={}
    ratingdict={}#每个用户各自的打分
    timeList=[]#每个用户对应每个物品的打分
    for  index2,userId,itemid,rating,timestamp in rating.values:
        #如果第一次循环
        #timeArray = time.strptime(timestamp, "%m/%d/%Y %H:%M")
        #转换为时间戳:
        #timestamp = int(time.mktime(timeArray))
        #userMovieDic={"user1":{"item1":[10d,100d,200d],"item2":[10d,20d]},[]}    
        timestamp=timestamp/3600/24
        #print("timeStamp",timestamp) 
        if preUserId==-1:
            
            timeList.append(timestamp)
            ratingdict[itemid]=timeList
            userMovieDic[userId]=ratingdict
        else:
            #如果接下来还是上一个用户的
            if userId==preUserId :
                if itemid in ratingdict.keys():#if user's item list already have this item
                    timeList=ratingdict[itemid]#append the time ti time list
                else: # user's item list do not have this item yet
                    timeList=[]#clear the time list
                timeList.append(timestamp)
                ratingdict[itemid]=timeList
                
            #如果接下来是新用户的
            if userId!=preUserId :
                
                
                #把上个用户的信息存到大字典里
                userMovieDic[preUserId]=ratingdict
                #清空上个用户信息
                ratingdict={}#清空
                timeList=[]
                #保存这个用户信息
                timeList.append(timestamp)
                ratingdict[itemid]=timeList                
                
        #赋当前用户名值给旧用户名
        preUserId=userId
        
    #print(userMovieDic)

    return userMovieDic
def drawPredict(): 
    x = np.arange(len(rPCList))
    #x = np.arange(20)
    y = timeSetList
    ppl.plot( x, y, linewidth=0.75)
    
    newrPCList=np.array(rPCList).reshape(52765,)
    y = newrPCList
    ppl.plot( x, y,'--', linewidth=0.75)
    plt.show()


## load data

In [6]:
"""load data"""
ratingTrain=pd.read_csv("processeddata/dataOrderMoreThan10TrainT.csv")
ratingTest=pd.read_csv("processeddata/dataOrderMoreThan10TestT.csv")
#ratingTrain=pd.read_csv("MovieLensMoreThan10TestData3/ratingsMl20MoreThan10TrainT.csv")
#ratingTest=pd.read_csv("MovieLensMoreThan10TestData3/ratingsMl20MoreThan10TestT.csv")

print("start load")
userMovieDicTraningset=loadUserRatingData(ratingTrain)
userMovieDicTestset=loadUserRatingData(ratingTest)
userMovieDicTraningTA=loadUserRatingDataTime(ratingTrain,"train")
userMovieDicTestsetTA=loadUserRatingDataTime(ratingTest,"test")

userMovieDicTraningQuantity=loadUserRatingQuantity(ratingTrain)
userMovieDicTestsetQuantity=loadUserRatingQuantity(ratingTest)
"""
userMovieDicTraningset=loadUserRatingDataFromMovL(ratingTrain)
userMovieDicTestset=loadUserRatingDataFromMovL(ratingTest)
userMovieDicTraningTA=loadUserRatingDataTimeFromMovL(ratingTrain)
userMovieDicTestsetTA=loadUserRatingDataTimeFromMovL(ratingTest)
"""
AllNeibors=getNeighborsFormFile()
print("end load")



start load
test time mean: 15235.3776682


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## test user based CF

In [7]:
TimeDelay=0
userBasedRecommendAll(AllNeibors,userMovieDicTraningset,5,10,True,userMovieDicTestset)

CF:
precision= 0.007488753638528712 rightNum 283 recommend 37790
recall= 0.024962512128429037 testSetNum 3779
F1= 0.011521159443890325
Novelt= 0.14323895210373114


## test time decay CF

In [8]:
TimeDelay=0
userBasedRecommendAllTDF(AllNeibors,userMovieDicTraningset,userMovieDicTraningTA,5,10,True,userMovieDicTestset,userMovieDicTestsetTA)







TDF:
precision= 0.006800740936755756 rightNum 257 recommend 37790
recall= 0.02266913645585252 testsetNum 3779
F1= 0.010462678364239624
Novelt= 0.13416247684572638
